In [26]:
import pandas as pd
import numpy as np

In [27]:
ww = pd.read_csv('C:\Asus WebStorage\psabin@gmail.com\MySyncFolder\Data Science Course\BlueBerry Winery\winequality-white.csv', sep=';')

In [28]:
rw = pd.read_csv('C:\Asus WebStorage\psabin@gmail.com\MySyncFolder\Data Science Course\BlueBerry Winery\winequality-red.csv', sep=';')

In [29]:
ww['wine_type'] = 'white'

In [30]:
rw['wine_type'] = 'red'

In [31]:
all_wine_ml = pd.concat([rw, ww])

In [32]:
skew_results_white_df = pd.DataFrame(columns=['feature', 'skew', 'kurtosis'])
ww_floats = ww.drop(['wine_type'], axis=1)

for feature in ww_floats.columns:
    skew = ww_floats[feature].astype(float).skew()
    kurtosis = ww_floats[feature].astype(float).kurt()
    new_row = [feature, skew, kurtosis]
    skew_results_white_df.loc[len(skew_results_white_df)] = new_row
    
print(skew_results_white_df) 

                 feature      skew   kurtosis
0          fixed acidity  0.647751   2.172178
1       volatile acidity  1.576980   5.091626
2            citric acid  1.281920   6.174901
3         residual sugar  1.077094   3.469820
4              chlorides  5.023331  37.564600
5    free sulfur dioxide  1.406745  11.466342
6   total sulfur dioxide  0.390710   0.571853
7                density  0.977773   9.793807
8                     pH  0.457783   0.530775
9              sulphates  0.977194   1.590930
10               alcohol  0.487342  -0.698425
11               quality  0.155796   0.216526


In [34]:
rw.drop(['wine_type'], axis=1, inplace=True)

In [35]:
ww.drop(['wine_type'], axis=1, inplace=True)

In [36]:
ww['quality_label'] = ww['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

# here we are transforming these labels into categrical data type (specific to pandas) instead of simple string
ww['quality_label'] = pd.Categorical(ww['quality_label'],
categories=['low', 'medium', 'high'])

In [37]:
rw['quality_label'] = rw['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

# here we are transforming these labels into categrical data type (specific to pandas) instead of simple string
rw['quality_label'] = pd.Categorical(rw['quality_label'],
categories=['low', 'medium', 'high'])

In [39]:
all_wine_ml.drop(['quality'], axis=1, inplace=True)

For binary values (red/white), use LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and 'color' is the column with 'red' or 'white'
le = LabelEncoder()
all_wine_ml['encoded_wine_type'] = le.fit_transform(all_wine_ml['wine_type'])  # Will encode 'red' as 0 and 'white' as 1
# I prefer to swap those values as so:
all_wine_ml['encoded_wine_type'] = 1 - all_wine_ml['encoded_wine_type']


Think this is wrong encoding for WW. Check later. Can undo by deleting and "run all".

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the column
encoded_quality = encoder.fit_transform(ww[['quality_label']])

# Create a DataFrame from the encoded column
encoded_df = pd.DataFrame(encoded_quality, columns=encoder.get_feature_names_out(['quality_label']))

# Concatenate the new DataFrame to the original one
ww = pd.concat([ww, encoded_df], axis=1)


In [46]:
display(all_wine_ml.head())

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,wine_type,encoded_wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,red,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,red,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,red,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,red,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,red,1


In [51]:
# We will use ALL FEATURES of all wine (except type) to predict type
wine_features = all_wine_ml.drop(['wine_type'], axis=1)
wine_type = wine_features.encoded_wine_type
wine_features.drop(['encoded_wine_type'],axis=1, inplace=True)
display(wine_features.head())
display(wine_type.head())

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


0    1
1    1
2    1
3    1
4    1
Name: encoded_wine_type, dtype: int32

Reshape the wine_type DataFrame to have 1 column instead of 0

In [52]:
wine_type = wine_type.values.reshape(-1, 1)
display(wine_type.shape)

(6497, 1)